# Classification for SENSE project


## SVM Classifier




In [2]:
# Import earthengine API
import ee

# Authenticate and initialise 
ee.Authenticate()
ee.Initialize()

import folium
!pip install geehydro # Life saver for plotting GEE stuff with Python!
import geehydro

  Created wheel for geehydro: filename=geehydro-0.2.0-py2.py3-none-any.whl size=10141 sha256=1446a5794f583484a4c7c9aca572e79f09a3b56d84ae80019c6e84208ac493f1
  Stored in directory: /root/.cache/pip/wheels/49/6c/92/019aeb37663078e6666afa9dd336e2d8df72d5683d7d357697
Successfully built geehydro


In [4]:
# importing Landsat 8 imegery (surface reflectance product)  
# l8 = ee.ImageCollection('LANDSAT/LC08/C01/T1')
dataset = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2');

aoi = ee.Geometry.Rectangle(77.47, 12.94, 77.51, 12.98)

dataset = dataset.filterBounds(aoi)

# Normalization
def normalize(image):
  bandNames = image.bandNames();

  # Compute min and max of the image
  minDict = image.reduceRegion(
    reducer = ee.Reducer.min(),
    geometry = aoi,
    scale = 30,
    maxPixels = 1e9,
    bestEffort = True,
    tileScale = 16
  )
  maxDict = image.reduceRegion(
    reducer = ee.Reducer.max(),
    geometry = aoi,
    scale = 30,
    maxPixels = 1e9,
    bestEffort = True,
    tileScale = 16
  );
  mins = ee.Image.constant(minDict.values(bandNames));
  maxs = ee.Image.constant(maxDict.values(bandNames));

  normalized = image.subtract(mins).divide(maxs.subtract(mins));
  return normalized;

ls_20 = normalize(dataset.filterDate('2020-01-01','2020-12-29').sort('CLOUD_COVER').first());
ls_19 = normalize(dataset.filterDate('2019-01-01','2019-12-29').sort('CLOUD_COVER').first());
ls_18 = normalize(dataset.filterDate('2018-01-01','2018-12-29').sort('CLOUD_COVER').first());

ls_17 = normalize(dataset.filterDate('2017-01-01','2017-12-29').sort('CLOUD_COVER').first());
ls_16 = normalize(dataset.filterDate('2016-01-01','2016-12-29').sort('CLOUD_COVER').first());
ls_15 = normalize(dataset.filterDate('2015-01-01','2015-12-29').sort('CLOUD_COVER').first());

ls_14 = normalize(dataset.filterDate('2014-01-01','2014-12-29').sort('CLOUD_COVER').first());
ls_13 = normalize(dataset.filterDate('2013-01-01','2013-12-29').sort('CLOUD_COVER').first());

# Use these bands for prediction
bands = ['SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B6', 'SR_B7', 'ST_TRAD'];


Importing the geometries


In [18]:
# Plot the result
water = ee.Geometry.MultiPolygon(
    [[[[77.33690896389112,12.97190512044105],
       [77.33896890041456,12.973159721940856],
       [77.33720937130079,12.974372497374777],
       [77.33690896389112,12.97190512044105]]],
     [[[77.35085645076856,12.974079759018426],
       [77.35085645076856,12.975083431948388],
       [77.34703698513135,12.973996119424758],
       [77.35085645076856,12.974079759018426]]],
     [[[77.3543755089961,12.977299861984198],
       [77.35523381588087,12.979725366550316],
       [77.35377469417676,12.979892641854967],
       [77.3543755089961,12.977299861984198]]],
     [[[77.3532597100459,12.97094325500556],
       [77.35386052486524,12.973954299617365],
       [77.35248723384962,12.973828840153],
       [77.3532597100459,12.97094325500556]]],
     [[[77.61773918531688,12.906125120213627],
       [77.61975620649608,12.910893830135821],
       [77.61683796308787,12.911981417904475],
       [77.61773918531688,12.906125120213627]]],
     [[[77.33417476730546,12.890126258496963],
       [77.33507598953446,12.893598445016687],
       [77.33147110061844,12.89330561186866],
       [77.33417476730546,12.890126258496963]]]])

non_urban = ee.Geometry.MultiPolygon(
    [[[[77.24945354417429,12.929421008943933],
       [77.25147056535349,12.932160681202756],
       [77.24814462617502,12.932390728649514],
       [77.24945354417429,12.929421008943933]]],
     [[[77.28891328183126,12.928465640334862],
       [77.28745416012715,12.92520307814124],
       [77.29402020779561,12.926499742580344],
       [77.28891328183126,12.928465640334862]]],
     [[[77.30126125411181,12.91305609917982],
       [77.30231268004565,12.915816865770896],
       [77.30029565886645,12.91633973479562],
       [77.30126125411181,12.91305609917982]]],
     [[[77.29313890214618,12.99528844415357],
       [77.29591767068561,12.996929725381255],
       [77.29415814157184,12.997943759952069],
       [77.29313890214618,12.99528844415357]]],
     [[[77.6682523410615,13.160411700453588],
       [77.67129933050242,13.161686238303005],
       [77.66108547857371,13.161623556268697],
       [77.6682523410615,13.160411700453588]]],
     [[[77.85864390266106,13.082716376130058],
       [77.85997427833244,13.08374050943408],
       [77.85765684974358,13.085830564200236],
       [77.85864390266106,13.082716376130058]]],
     [[[77.91360275741509,13.07310773259363],
       [77.91512625213555,13.07609663432832],
       [77.90783064361504,13.076389252132401],
       [77.91360275741509,13.07310773259363]]],
     [[[77.4205055925501,12.702860886609601],
       [77.41213710042364,12.698590627922417],
       [77.42895991536504,12.697920776914748],
       [77.4205055925501,12.702860886609601]]],
     [[[77.75572558363297,12.954987790766982],
       [77.7562191100917,12.953063925428436],
       [77.76121874769547,12.956472502771119],
       [77.75572558363297,12.954987790766982]]],
     [[[77.36095833992466,12.953696423312978],
       [77.36662316536412,12.96406836459562],
       [77.35855508064732,12.96490479264407],
       [77.36095833992466,12.953696423312978]]],
     [[[77.39494729256138,12.988825444274283],
       [77.39923882698521,12.989829057657177],
       [77.40009713386998,12.995014328859492],
       [77.39237237190709,12.993341672560065],
       [77.39494729256138,12.988825444274283]]],
     [[[77.43923592781529,12.946335428759095],
       [77.44421410774693,12.951688900911062],
       [77.43803429817662,12.95503476259268],
       [77.43923592781529,12.946335428759095]]],
     [[[77.34327721809849,12.999195920303883],
       [77.34756875252232,12.990330862827909],
       [77.35409188484654,12.997857818705771],
       [77.34293389534459,13.000366753284327],
       [77.34327721809849,12.999195920303883]]],
     [[[77.88211625784514,12.912035541695577],
       [77.8875665065634,12.916134867547562],
       [77.87979882925627,12.919355719248044],
       [77.88211625784514,12.912035541695577]]],
     [[[77.84628194540618,12.908521780304845],
       [77.84937185019133,12.91320678451529],
       [77.83997338980315,12.913792403866662],
       [77.84628194540618,12.908521780304845]]],
     [[[77.63245876734034,13.192779196062471],
       [77.63677175943629,13.195474165899727],
       [77.63110693399683,13.196894758192853],
       [77.63245876734034,13.192779196062471]]],
     [[[77.61928375665919,13.20046709348754],
       [77.62263115350977,13.206483540099613],
       [77.61670883600489,13.206399979356197],
       [77.61928375665919,13.20046709348754]]],
     [[[77.65902336542383,13.187180249011073],
       [77.66288574640528,13.195620403722986],
       [77.65704925958887,13.195453274774978],
       [77.65902336542383,13.187180249011073]]],
     [[[77.59456451837794,13.204728758482483],
       [77.59782608454005,13.207319145960899],
       [77.59293373529688,13.208656109390175],
       [77.59456451837794,13.204728758482483]]],
     [[[77.6038342327334,13.178405521881848],
       [77.60735329096094,13.18208258870698],
       [77.60228928034083,13.183753964448005],
       [77.6038342327334,13.178405521881848]]]])

urban = ee.Geometry.MultiPolygon(
    [[[[77.5562377152835,12.939736499257096],
       [77.55645765642272,12.939689445500447],
       [77.55658103803741,12.940782136002348],
       [77.55633963922607,12.940803048692528],
       [77.5562377152835,12.939736499257096]]]])

SVM Classifier

In [22]:
NB = ee.Classifier.libsvm(
  kernelType = 'RBF',
  gamma = 0.5,
  cost = 10)

# Combine polygons for each class

polygons = ee.FeatureCollection([
  ee.Feature(urban, {'class': 0}),
  ee.Feature(water, {'class': 1}),
  ee.Feature(non_urban, {'class': 2}),
])

#  Sample rasters from each image to create training data

training_20 = ls_20.sampleRegions(
  collection = polygons,
  properties = ['class'],
  scale = 30
  );

training_19 = ls_19.sampleRegions(
  collection = polygons,
  properties = ['class'],
  scale = 30
  );

training_18 = ls_18.sampleRegions(
  collection = polygons,
  properties = ['class'],
  scale = 30
  );

training_17 = ls_17.sampleRegions(
  collection = polygons,
  properties = ['class'],
  scale = 30
  );

training_16 = ls_16.sampleRegions(
  collection = polygons,
  properties = ['class'],
  scale = 30
  );

training_15 = ls_15.sampleRegions(
  collection = polygons,
  properties = ['class'],
  scale = 30
  );

training_14 = ls_14.sampleRegions(
  collection = polygons,
  properties = ['class'],
  scale = 30
  );

training_13 = ls_13.sampleRegions(
  collection = polygons,
  properties = ['class'],
  scale = 30
  );

#  Train the classifier
trained_20 = NB.train(training_20, 'class',bands);
trained_19 = NB.train(training_19, 'class',bands);
trained_18 = NB.train(training_18, 'class',bands);

trained_17 = NB.train(training_17, 'class',bands);
trained_16 = NB.train(training_16, 'class',bands);
trained_15 = NB.train(training_15, 'class',bands);

trained_14 = NB.train(training_14, 'class',bands);
trained_13 = NB.train(training_13, 'class',bands);

#  Classify the images
classified_20 = ls_20.classify(trained_20);
classified_19 = ls_19.classify(trained_19);
classified_18 = ls_18.classify(trained_18);

classified_17 = ls_17.classify(trained_17);
classified_16 = ls_16.classify(trained_16);
classified_15 = ls_15.classify(trained_15);

classified_14 = ls_14.classify(trained_14);
classified_13 = ls_13.classify(trained_13);


Outputs

In [23]:
# Plot the result
# classified_reduced = classified.clip(roi)
map = folium.Map(location=[12.94, 77.47], zoom_start=11)

map.addLayer(classified_20,{'min': 0, 'max': 1, 'palette' :['red','blue','green']},'classification 2020');
map.addLayer(classified_19,{'min': 0, 'max': 1, 'palette' :['red','blue','green']},'classification 2019');
map.addLayer(classified_18,{'min': 0, 'max': 1, 'palette' :['red','blue','green']},'classification 2018');

map.addLayer(classified_17,{'min': 0, 'max': 1, 'palette' :['red','blue','green']},'classification 2017');
map.addLayer(classified_16,{'min': 0, 'max': 1, 'palette' :['red','blue','green']},'classification 2016');
map.addLayer(classified_15,{'min': 0, 'max': 1, 'palette' :['red','blue','green']},'classification 2015');

map.addLayer(classified_14,{'min': 0, 'max': 1, 'palette' :['red','blue','green']},'classification 2014');
map.addLayer(classified_13,{'min': 0, 'max': 1, 'palette' :['red','blue','green']},'classification 2013');

folium.LayerControl().add_to(map)

map


## Unsupervised classification

The `ee.Clusterer` package handles unsupervised classification (or clustering) in Earth Engine. More details about each Clusterer are available in the [reference docs in the Code Editor](https://code.earthengine.google.com/#workspace).

Clusterers are used in the same manner as classifiers in Earth Engine. The general workflow for clustering is:

*  Assemble features with numeric properties in which to find clusters.
*   Instantiate a clusterer. Set its parameters if necessary.
*  Train the clusterer using the training data.
* Apply the clusterer to an image or feature collection.
* Label the clusters.


The training data is a `FeatureCollection` with properties that will be input to the clusterer. 

Unlike classifiers, there is no input class value for a Clusterer. 

Like classifiers, the data for the train and apply steps are expected to have the same number of values. When a trained clusterer is applied to an image or table, it assigns an integer cluster ID to each pixel or feature.

These algorithms are currently based on the algorithms with the same name in [Weka](https://www.cs.waikato.ac.nz/ml/weka/).

Here is a simple example of building and using an `ee.Clusterer`:

In [ ]:
# Load a pre-computed Landsat composite for input.
input = ee.Image('LANDSAT/LE7_TOA_1YEAR/2001')

# Define a region in which to generate a sample of the input.
region = ee.Geometry.Rectangle(29.7, 30, 32.5, 31.7)

# Display the sample region.

map = folium.Map(location=[31,31.5],zoom_start=8)

map.addLayer(ee.Image().paint(region, 0, 2), {}, 'region')
map

In [ ]:
# Make the training dataset.
training = input.sample(region = region,scale= 30,  numPixels= 5000)

# Instantiate the clusterer and train it.
clusterer = ee.Clusterer.wekaKMeans(15).train(training)

# Cluster the input using the trained clusterer.
result = input.cluster(clusterer)

# Display the clusters with random colors.

map.addLayer(result.randomVisualizer(), {}, 'clusters')
folium.LayerControl().add_to(map)
map
